In [13]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm

# Load dataset (change to test or valid as needed)
df = pd.read_csv("/Users/arushijain/PycharmProjects/AIPoweredCodeAssistant/datasets/security/test_preprocessed.csv")

# Select a sample to avoid running on full dataset immediately
sample_df = df[['id', 'func', 'target']].sample(20, random_state=42)

# Store LLM predictions
llm_outputs = []

# Define the prompt template
def create_prompt(code_snippet):
    return f"""
You are a security auditor. Analyze the following C code and determine whether it contains any security vulnerabilities (such as buffer overflow, memory leaks, use-after-free, or unsafe pointer manipulation). If yes, explain what they are. If not, state it's safe.

Code:
{code_snippet}

Answer with one of the following:
- 'VULNERABLE' and the reason
- 'SAFE' and why
"""

# Query OpenAI
def get_llm_response(prompt):
    try:
        client = OpenAI(api_key="" )
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[{"role": "user", "content": prompt}],
            temperature = 0
        )
        completion = response.choices[0].message.content
        return completion
    except Exception as e:
        return f"ERROR: {e}"

# Run LLM over the sample
for _, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
    code = row['func']
    target = row['target']
    prompt = create_prompt(code)
    response = get_llm_response(prompt)

    # Parse response for classification
    pred = "VULNERABLE" if "vulnerable" in response.lower() else "SAFE"

    llm_outputs.append({
        "id": row['id'],
        "true_label": "VULNERABLE" if target else "SAFE",
        "llm_pred": pred,
        "response": response
    })

# Create result DataFrame
results_df = pd.DataFrame(llm_outputs)

# Save results for further analysis
results_df.to_csv("llm_vulnerability_results.csv", index=False)

# Optional: Print accuracy
accuracy = (results_df['true_label'] == results_df['llm_pred']).mean()
print(f"\nLLM Classification Accuracy: {accuracy:.2f}")


100%|██████████| 20/20 [02:38<00:00,  7.91s/it]


LLM Classification Accuracy: 0.75


In [15]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(results_df['true_label'], results_df['llm_pred']))
print(classification_report(results_df['true_label'], results_df['llm_pred']))

[[9 4]
 [1 6]]
              precision    recall  f1-score   support

        SAFE       0.90      0.69      0.78        13
  VULNERABLE       0.60      0.86      0.71         7

    accuracy                           0.75        20
   macro avg       0.75      0.77      0.74        20
weighted avg       0.80      0.75      0.76        20

